## Creating the ball_chaser ROS Package

The second major task in this project is to create the `ball_chaser` ROS package. Within this package, you'll analyze the image captured by the camera to determine the position of a white ball. Then you’ll drive the robot toward the ball. The nodes in `ball_chaser` will communicate with the `my_robot` package by subscribing to the robot camera sensor and publishing to the robot’s wheel joints.

---

### Package Nodes

The `ball_chaser` package will have two C++ nodes:

1. **`drive_bot`:**
   - This server node will provide a `ball_chaser/command_robot` service to drive the robot around by controlling its linear x and angular z velocities.
   - The service will publish a message containing the velocities to the wheel joints.

2. **`process_image`:**
   - This client node will subscribe to the robot’s camera images and analyze each image to determine the position of the white ball.
   - Once the ball’s position is determined, the client node will request a service to drive the robot either left, right, or forward.

---

### Drive Bot Server Node

#### Writing the ROS Service File

1. **Create the `DriveToTarget.srv` File**
   - Navigate to the `srv` directory of the `ball_chaser` package and create the service file:

     ```bash
     $ cd ~/catkin_ws/src/ball_chaser/srv
     $ touch DriveToTarget.srv
     ```

   - Define the `DriveToTarget.srv` file as follows:

     **Request:**
     ```
     float64 linear_x
     float64 angular_z
     ```
     **Response:**
     ```
     string msg_feedback
     ```

2. **Test the Service**
   - After writing the service file, test it with ROS:

     ```bash
     $ cd ~/catkin_ws/
     $ source devel/setup.bash
     $ rossrv show DriveToTarget
     ```

   - You should see:

     ```
     [ball_chaser/DriveToTarget]:
     float64 linear_x
     float64 angular_z
     ---
     string msg_feedback
     ```

#### Writing the `drive_bot.cpp` Node

Create the `drive_bot.cpp` file under the `src` directory of the `ball_chaser` package. Use the following structure:

```cpp
#include "ros/ros.h"
#include "ball_chaser/DriveToTarget.h"
#include "geometry_msgs/Twist.h"

// ROS::Publisher motor commands
ros::Publisher motor_command_publisher;

// Callback function for the service
bool handle_drive_request(ball_chaser::DriveToTarget::Request& req,
                          ball_chaser::DriveToTarget::Response& res) {
    ROS_INFO("DriveToTargetRequest received - linear_x:%1.2f, angular_z:%1.2f", (float)req.linear_x, (float)req.angular_z);

    // Publish the requested linear and angular velocities
    geometry_msgs::Twist motor_command;
    motor_command.linear.x = req.linear_x;
    motor_command.angular.z = req.angular_z;
    motor_command_publisher.publish(motor_command);

    // Return a response message
    res.msg_feedback = "Wheel velocities set - linear_x: " + std::to_string(req.linear_x) + ", angular_z: " + std::to_string(req.angular_z);
    ROS_INFO_STREAM(res.msg_feedback);

    return true;
}

int main(int argc, char** argv) {
    // Initialize the drive_bot node
    ros::init(argc, argv, "drive_bot");
    ros::NodeHandle n;

    // Define a publisher for motor commands
    motor_command_publisher = n.advertise<geometry_msgs::Twist>("/cmd_vel", 10);

    // Define a service with a callback function
    ros::ServiceServer service = n.advertiseService("/ball_chaser/command_robot", handle_drive_request);
    ROS_INFO("Ready to send wheel commands");

    // Spin to handle callbacks
    ros::spin();

    return 0;
}
```

---

### Update `CMakeLists.txt`

After writing the server node, update the `CMakeLists.txt` file:

1. **Add Dependencies:**

   ```cmake
   add_compile_options(-std=c++11)

   add_service_files(
     FILES
     DriveToTarget.srv
   )

   generate_messages(
     DEPENDENCIES
     std_msgs
   )

   include_directories(
     include
     ${catkin_INCLUDE_DIRS}
   )

   add_executable(drive_bot src/drive_bot.cpp)
   target_link_libraries(drive_bot ${catkin_LIBRARIES})
   add_dependencies(drive_bot ball_chaser_generate_messages_cpp)
   ```

2. **Build the Package:**

   ```bash
   $ cd ~/catkin_ws/
   $ catkin_make
   ```

---

### Testing the `drive_bot.cpp` Node

1. **Launch the Robot in Gazebo:**

   ```bash
   $ cd ~/catkin_ws/
   $ source devel/setup.bash
   $ roslaunch my_robot world.launch
   ```

2. **Run the `drive_bot` Node:**

   ```bash
   $ cd ~/catkin_ws/
   $ source devel/setup.bash
   $ rosrun ball_chaser drive_bot
   ```

3. **Request the Service:**

   Open a new terminal and test the service by sending velocity commands:

   ```bash
   $ cd ~/catkin_ws/
   $ source devel/setup.bash
   $ rosservice call /ball_chaser/command_robot "linear_x: 0.5
   angular_z: 0.0"  # Forward

   $ rosservice call /ball_chaser/command_robot "linear_x: 0.0
   angular_z: 0.5"  # Left

   $ rosservice call /ball_chaser/command_robot "linear_x: 0.0
   angular_z: -0.5"  # Right

   $ rosservice call /ball_chaser/command_robot "linear_x: 0.0
   angular_z: 0.0"  # Stop
   ```

---

### Launch File for `drive_bot`

Create a `ball_chaser.launch` file under the `launch` directory of the `ball_chaser` package:

```xml
<launch>
  <!-- The drive_bot node -->
  <node name="drive_bot" type="drive_bot" pkg="ball_chaser" output="screen"/>
</launch>
```
